# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

# TO DO (ctrl+f `!!!`):
- Add Day 3 slides (this cell)
- Check model update equation (Section 3)
- Explain difference between sensitivity kernel, misfit kernel, event kernel, gradient (section 3)



## Part 3b: Seismic Imaging (advanced)

- In this notebook we perform a multi-event, multi-station 2D inversion
- We will perform all book keeping tasks manually
- At the end of the notebook, we perform the same inversion using SeisFlows
- **Objective**: Illustrate the laborious book keeping required as inversion problems scale
- These instructions should be run from inside the Docker container, using Jupyter Lab (see *Docker Preamble* in Day 0). 

-----------

**Relevant Links:** 
- Day 3 Slides !!! ADD THIS !!!
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_3a_imaging_adv.ipynb
- Completed Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_3a_imaging_adv.ipynb
- Day 0 Notebook (Container Testing): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Day 1A Notebook (Intro SPECFEM): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1a_intro_specfem2d.ipynb
- Day 1B Notebook (Fwd. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb
- Day 2A Notebook (Adj. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2a_kernels.ipynb
- Day 3A Notebook (Simple Imaging): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_3a_imaging_simple.ipynb


**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.

----------------

# A Multi-event, Multi-station Inversion

- Real seismic inversions use multiple events recorded at multiple stations to increase coverage and reduce nonuniqueness
- Here we attempt a *multi-event* ($N=10$), multi-station ($S=10$) model update with the skills we have picked up in previous notebooks
- The inversion workflow is as follows: 
    0) **data**: run $N$ forward simulations to generate *data* using target model ($m_{true}$) for $N$ events and $S$ stations
    1) **synthetics**: run $N$ forward simulations to generate synthetics through starting model ($m_{init}$) for $N$ events and $S$ stations
    2) **misfit**: quantify data-synthetic misfit ($\chi_{init}$) and generate adjoint sources for each source-receiver pair ($N\times S$ waveforms)
    3) **kernels**: generate misfit kernels with $N$ adjoint simulation
    4) **update**: scale gradient and update initial model to get trial model: $m_{try}$
    5) **line search**: run $N$ forward simulations using trial model $m_{try}$ to get $N \times S$ new synthetic waveforms
    6) **line search**: calculate misfit ($\chi_{try}$) and determine if misfit is reduced ($\chi_{try} < \chi_{init}$?)
    7) **line search**: if misfit is not reduced ($\chi_{try} >=  \chi_{init}$), repeat steps 5-7 with newly scaled gradient
    8) **line search**: if misfit is reduced, accept trial model as newly updated model ($m_{try} \rightarrow m_{01}$)
    9) **finalize**: set newly updated model as initial model ($m_{01} \rightarrow m_{init}$) and repeat steps 1-9
    



In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from IPython.display import Image
from scipy.integrate import simps
from seisflows.tools.specfem import Model, read_fortran_binary, write_fortran_binary

## 1) Set Up a SPECFEM2D Working Directory

- We want to set up a clean working directory to run SPECFEM2D inside. This will help us preserve our cloned repository and reduce file clutter.
- 10 sources, 10 receivers
- Starting model: homogeneous halfspace
- Target model: perturbation checkerboard.
- **Objective**: Generate a misfit kernel and manually update a 2D model, quantify misfit 
- **NOTE:** We will work in `/home/scoped/work/day_3/inversion_example`. The following cells assume relative paths, so you must evaluate the '%cd' command below.

In [ ]:
# Make sure we're in an empty working directory
! rm -rf /home/scoped/work/day_3/inversion_example
! mkdir -p /home/scoped/work/day_3/inversion_example
%cd /home/scoped/work/day_3/inversion_example

# Setup SPECFEM2D working directory
! ln -s /home/scoped/specfem2d/bin .
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
! mkdir OUTPUT_FILES

## 2) Target Model (checkerboard) Synthetics (n=100)

- We use the Target checkerboard model used to generate 'data'
- 10 sources and 10 stations with pre-defined locations
- Set the Par_file to output Model files in Fortran binary which we will use for visualizations

In [ ]:
%cd /home/scoped/work/day_3/inversion_example

# Set up the Par_file
! cp -f DATA/Par_file_Tape2007_132rec_checker DATA/Par_file

! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.
! seisflows sempar -P DATA/Par_file NSTEP 5000  # 4800 -> 5000 to match the other Par_file

# Ensure that we're using stations 1-10
! head -10 DATA/STATIONS_checker > DATA/STATIONS  

# Ensure that SPECFEM can find the checkerboard model by naming it correctly
! cp -f DATA/model_velocity.dat_checker DATA/proc000000_model_velocity.dat_input

### a) Visualize Experimental Setup

- We plot the 10 sources and 10 receivers with text labels
- Also plot the underlying checkerboard model to see where the perturbatiosn are
- The initial model is a homogeneous halfspace and is **not** plotted here

In [ ]:
def plot_sources_receivers(nevents=10):
    """
    Small code block to plot source-receiver geometry with text labels
    Assumes relative pathing
    """
    # Grab Station locations
    sta_x, sta_z = np.genfromtxt("DATA/STATIONS", dtype=float, usecols=[2, 3]).T
    
    # Grab Event locations
    ev_x, ev_z = [], []
    for i in range(1, nevents+1):
        source_file = f"DATA/SOURCE_{i:0>3}"
        with open(source_file, "r") as f:
            lines = f.readlines()
        # Trying to break apart the following line
        # 'xs = 299367.72      # source location x in meters\n'
        xs = float(lines[2].split("=")[1].split("#")[0].strip())
        zs = float(lines[3].split("=")[1].split("#")[0].strip())

        ev_x.append(xs)
        ev_z.append(zs)
        
    # Plot SOURCES and STATIONS geometry
    for i, (x, z) in enumerate(zip(ev_x, ev_z)):
        plt.scatter(x, z, c="y", marker="*", edgecolor="k", s=50)
        plt.text(x, z, f"S{i+1:0>2}", fontsize=9, c="g")  # SOURCE numbering starts at 1
    for i, (x, z) in enumerate(zip(sta_x, sta_z)):
        plt.scatter(x, z, c="c", marker="v", s=40, edgecolor="k")
        plt.text(x, z, f"R{i:0>2}", fontsize=9)
        
    # Finalize plot labels
    plt.xlabel("X [m]")
    plt.ylabel("Z [m]")
    plt.xlim([0, 480E3])
    plt.ylim([0, 480E3])
    plt.title("SOURCE-RECEIVER GEOMETRY")
    
    
# Load and plot checkerboard model
data = np.genfromtxt("DATA/model_velocity.dat_checker", dtype=float, usecols=[1,2,4,5])
chkbd_x, chkbd_z, chkbd_vp, chkbd_vs = data.T
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vp, levels=125, cmap="seismic_r", alpha=0.1)

# Plot sources and receivers on top
plot_sources_receivers()
plt.show()

### b) Running the Forward Solver

- We need to run the forward solver once per event (i.e., 10 times)
- Each time we run SPECFEM, it overwrites existing files in the OUTPUT_FILES/ directory
- In order to retain files, we need to do some file management after each forward simulation
- We use Python for control flow but this can easily be accomplished using other languages
- We define a re-usable function below to take care of looping through events and book keeping
- **NOTE**: We really only have to run the mesher once to set up the database. Here we do not, but in 3D you will want to skip that step.

In [ ]:
def run_fwd_mesher_solver(ntask, save_synthetics, save_forward=False):
    """
    Run the mesher and solver `ntask` times. Save the resulting synthetics
    to path `save_synthetics`. Assumes relative pathing.
        
    :type ntask: int
    :param ntask: number of sources to run
    :type save_synthetics: str
    :param save_synthetics: relative directory name to save synthetics to
    """
    print(f"Generating synthetics for {ntask} sources")
    
    # Run `ntask` forward simulations using Target model, generating 'data' for S stations. 
    for i in range(1, ntask + 1):
        # Select the task/source to run mesher/sovler with
        source_fid = f"SOURCE_{i:0>3}"
        source_path = f"DATA/{source_fid}"
        print(f"Running mesher and solver for {source_fid}")

        # Set the correct source for SPECFEM to find
        if os.path.exists("DATA/SOURCE"):
            os.remove("DATA/SOURCE")
        shutil.copy(source_path, "DATA/SOURCE")

        # Run xmeshfem and xspecfem 
        ! mpirun -n 1 bin/xmeshfem2D > OUTPUT_FILES/output_mesher.txt
        ! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/output_solver.txt

        # Move the just-generated synthetic waveforms so they don't get overwritten
        synthetics_dir = os.path.join(source_fid, save_synthetics)
        if os.path.exists(synthetics_dir):
            shutil.rmtree(synthetics_dir)
        os.makedirs(synthetics_dir)
        for src in glob("OUTPUT_FILES/*.semd"):
            dst = os.path.join(source_fid, save_synthetics, os.path.basename(src))
            os.rename(src, dst)
        
        # Save forward arrays stored on disk if requested
        if save_forward:
            print(f"saving forward arrays {source_fid}")
            save_fwd_dir = os.path.join(source_fid, "SAVE_FORWARD")
            if not os.path.exists(save_fwd_dir):
                os.makedirs(save_fwd_dir)
            # Save fwd arrays are the last frame and boundary conditions
            fid_templates = ["lastframe_elastic*.bin", "absorb_elastic_*.bin"]
            for fid in fid_templates:
                for src in glob(f"OUTPUT_FILES/{fid}"):
                    dst = os.path.join(save_fwd_dir, os.path.basename(src))
                    os.rename(src, dst)
        
    print("Finished generating synthetics")

In [ ]:
run_fwd_mesher_solver(ntask=10, save_synthetics="TARGET", save_forward=False)

## 3) Initial Model Synthetics

- Now we'll generate synthetics through a homogeneous halfspace initial model
- **IMPORTANT**: We need to save the forward wavefield (parameter `SAVE_FORWARD`) from the forward simulation of each $N$ events.
- The saved forward wavefield is a snapshot of the forward wavefield at the last time step
- The saved forward wavefield is used to reconstruct the forward wavefield during an adjoint simulation
- For large domains (and especially in 3D), saved forward wavefields are **large** files (on the order of GB)
- It is *not* preferable to save these large files for extended periods of time
- We will follow that practice here by deleting the forward wavefield after each adjoint simulation

In [ ]:
%cd /home/scoped/work/day_3/inversion_example

# Re-set the SPECFEM2D Par_file which is set up for the Initial model
! cp -f DATA/Par_file_Tape2007_onerec DATA/Par_file

# Edit Par_file to get the correct output files for Initial model
! seisflows sempar -P DATA/Par_file save_model binary
! seisflows sempar -P DATA/Par_file setup_with_binary_database 1
! seisflows sempar -P DATA/Par_file use_existing_stations .true.
! seisflows sempar -P DATA/Par_file save_forward .true.

# Run mesher and solver for the Target model and 10 sources. 
# Also save forward arrays for subsequent adjoint simulations
run_fwd_mesher_solver(ntask=10, save_synthetics="SYNTHETIC", save_forward=True)

## 4) Quantify Misfit, Generate Adjoint Sources for Initial Model

- We want to quantify misfit for all $N \times S$ waveforms
- We also want to generate $N \times S$ adjoint sources for subsequent adjoint simulations
- We'll use a waveform difference misfit for simplicity
- Misfit: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$
- Adjoint Source: $f^\dagger (t) = s(t) - d(t)$
- We'll define a re-usable Python function to calculate misfit and generate adjoint sources

In [ ]:
def quantify_misfit_single(data_fid, syn_fid): 
    """
    Quantify misfit and generate adjoint source w/ waveform difference
    For a single source-receiver pair
    
    :type data_fid: str
    :param data_fid: filename and path to data array
    :type syn_fid: str
    :param syn_fid: filename and path to synthetic array
    """
    # Load data into NumPy arrays
    t_data, d_data = np.loadtxt(data_fid).T
    t_syn, d_syn = np.loadtxt(syn_fid).T
    
    # dt represents the time step
    dt = t_syn[1] - t_syn[0] 
    
    # Calculate waveform difference misfit
    misfit = 1/2 * simps((d_data - d_syn)**2, dx=dt)
    
    # Create the time-amplitude array for adjoint source
    d_adj = d_data - d_syn
    adj_src = np.vstack((t_syn, d_adj)).T
    
    return misfit, adj_src    


def quantify_misfit(misfit_file, write_adjsrc, syn_dir="SYNTHETIC"):
    """
    Quantify Misfit for all sources available
    
    :type write_misfit: str
    :param write_misfit: file name to write misfit to
    :type write_adjsrc: bool
    :param write_adjsrc: choose whether or not to save adjoint sources
    :type syn_dir: str
    :param syn_dir: relative path to directory where synthetics are stored
    """
    for src in sorted(glob("SOURCE_???")):
        print(f"Quantifying misfit for {src}")

        # Set up directories to store misfit and adjoint sources
        # Make sure we're writing to fresh files/directories
        misfit_path = os.path.join(src, misfit_file)
        if os.path.exists(misfit_path):
            os.remove(misfit_path)
                
        if write_adjsrc:
            adj_src_dir = os.path.join(src, "ADJOINT_SOURCES")
            if os.path.exists(adj_src_dir):
                shutil.rmtree(adj_src_dir)
            os.makedirs(adj_src_dir)

        # Loop through each of the synthetic waveforms to get file names
        for syn_fid in sorted(glob(os.path.join(src, syn_dir, "*"))):
            # e.g., AA.S000001.BXY.semd
            fid_base = os.path.basename(syn_fid)
            data_fid = os.path.join(src, "TARGET", fid_base)
            
            misfit, adj_src = quantify_misfit_single(data_fid, syn_fid)
            print(f"\t{fid_base}; MISFIT={misfit:.3E}")
            
            # Write misfit to file
            with open(misfit_path, "a") as f:
                f.write(f"{misfit}\n")

            # Write adjoint source to file
            # Note that SPECFEM expects adjoint sources to be suffixed '.adj.
            if write_adjsrc:
                np.savetxt(os.path.join(adj_src_dir, fid_base.replace("semd", "adj")), adj_src)
                
def plot_waveforms(t, data, labels, colors):
    """Re-usable function to plot synthetic waveforms"""
    for i, d in enumerate(data):
        plt.plot(t, d, c=colors[i], label=labels[i])
    plt.xlabel("Time [s]")
    plt.ylabel("Displacement [m]")
    plt.legend()  

In [ ]:
# Run misfit quantification for initial model
quantify_misfit(misfit_file="MISFIT_INIT.txt", write_adjsrc=True, synthetic_dir="SYNTHETIC")

#### Understanding misfit quantification
- For each of our $N$ sources we have quantified misfit for $S$ stations
- Each synthetic has a corresponding adjoint source, which is stored in `SOURCE_???/ADJOINT_SOURCE`
- Each adjoint source is equal to the waveform difference between data and synthetic
- Each adjoint source has a corresponding misfit value
- Misfit values are stored in a text file which can be summed to calculate the overall **event misfit**
- The adjoint sources will be used to run $N$ adjoint simulations
- The misfit value will be used to determine if misfit is reduced during the line search

In [ ]:
# We can take a look at the files we just created
! ls SOURCE_001
! echo 
! echo "> ADJOINT SOURCES"
! ls SOURCE_001/ADJOINT_SOURCES
! echo 
! echo "> SOURCE_001/ADJOINT_SOURCES/AA.S000000.BXY.adj"
! head SOURCE_001/ADJOINT_SOURCES/AA.S000000.BXY.adj
! echo
! echo "> SOURCE_001/MISFIT_INIT.txt"
! head SOURCE_001/MISFIT_INIT.txt

In [ ]:
# Plot an example of data, synthetic and adjoint source
t_init, d_init = np.loadtxt("SOURCE_001/SYNTHETIC/AA.S000000.BXY.semd").T
t_true, d_true = np.loadtxt("SOURCE_001/TARGET/AA.S000000.BXY.semd").T
t_adjs, d_adjs = np.loadtxt("SOURCE_001/ADJOINT_SOURCES/AA.S000000.BXY.adj").T

plot_waveforms(t=t_init, data=[d_init, d_true, d_adjs],
               labels=["initial", "target", "adjsrc"],
               colors=["r", "k", "g"])

plt.title("Initial vs. Target Model Synthetics")
plt.show()

In [ ]:
plot_waveforms(t=t_init, data=[d_init, d_true, d_adjs],
               labels=["initial", "target", "adjsrc"],
               colors=["r", "k", "g"])
plt.xlim([45,145])
plt.title("Initial vs. Target Model Synthetics")
plt.show()

## 5) Run Adjoint Simulations for Event Kernels

- Now we need to run the adjoint simulations to generate event kernels
- Event kernels tell us what parts of the model are sensitive to the misfit function for **EACH** event
- Adjoint simulations take $S$ adjoint sources as input 
- **NOTE**: We need to place the adjoint sources in the correct location and also tell SPECFEM that we are running an adjoint simulation
- SPECFEM expects adjoint sources in the `SEM/` directory, and formatted exactly the same as its output synthetics
- We define a Python function below to run the adjoint solver and take care of book keeping for adjoint sources and kernels

In [ ]:
def run_adj_solver():
    """
    Counterpart to `run_fwd_mesher_solver`, runs adjoint simulations for `ntask` events
    and saves the resulting kernels. Assumes relative paths
    """
    ntask = len(glob("SOURCE_???"))
    print(f"Running adjoint simulations for {ntask} events")
    
    for i in range(1, ntask + 1):
        # Choose the task/source to run mesher/sovler with
        source_fid = f"SOURCE_{i:0>3}"
        source_path = f"DATA/{source_fid}"
        print(f"Running adjoint simulation for {source_fid}")

        # Set the correct source for SPECFEM to find
        if os.path.exists("DATA/SOURCE"):
            os.remove("DATA/SOURCE")
        shutil.copy(source_path, "DATA/SOURCE")
                
        # Symlink adjoint sources so SPECFEM can find them
        if os.path.islink("SEM"):
            os.remove("SEM")
        os.symlink(os.path.join(source_fid, "ADJOINT_SOURCES"), "SEM")
        
        # Copy the saved forward array so that SPECFEM can find it
        for src in glob(os.path.join(source_fid, "SAVE_FORWARD", "*")):
            dst = os.path.join("OUTPUT_FILES", os.path.basename(src))
            if os.path.exists(dst):
                os.remove(dst)
            shutil.copy(src, dst)
            
        # Run the adjoint simulation
        ! mpirun -n 1 bin/xspecfem2D > OUTPUT_FILES/adjoint_solver.txt
        
        # Move the resulting event kernel to disk
        kernel_dir = os.path.join(source_fid, "KERNELS")
        if os.path.exists(kernel_dir):
            shutil.rmtree(kernel_dir)
        os.makedirs(kernel_dir)
        for src in glob("OUTPUT_FILES/*kernel.bin"):
            dst = os.path.join(kernel_dir, os.path.basename(src))
            os.rename(src, dst)
            
    print("Finished")

In [ ]:
# Set up the Par_file to run an adjoint simulation (type 3) 
# and output kernels in expected output (FORTRAN binary)
! seisflows sempar -P DATA/Par_file simulation_type 3
! seisflows sempar -P DATA/Par_file save_ASCII_kernels .false.

# Run adjoint simluation for each N sources. This may take a while
run_adj_solver()

## 6) Sum Event Kernels and Scale Gradient

- Now we have $N$=10 event kernels, each representing misfit sensitivity for each of the $N$ events
- We need to sum the *event kernels* into a *misfit kernel*, which represents misfit sensitivity for **all** $N$ events
- We can use the `xcombine_sem` executable to sum these kernels
- We will also need to scale the misfit kernel before perturbing the initial model


In [ ]:
# First we need to tell SPECFEM where all our kernel files are by 
# listing paths in an input file
with open("kernel_paths.txt", "w") as f:
    for fid in sorted(glob("SOURCE_???/KERNELS")):
        f.write(f"{os.path.abspath(fid)}\n")

# Make a directory to store gradient files
! mkdir GRADIENT

# mpirun -n NPROC bin/xcombine_sem KERNEL_NAMES INPUT_FILE OUTPUT_DIR*
! mpirun -n 1 bin/xcombine_sem alpha_kernel,beta_kernel kernel_paths.txt GRADIENT 

# Rename 'alpha' to Vp and 'beta' to Vs for easier name recognition
for src in glob("GRADIENT/*"):
    if "alpha" in src:
        os.rename(src, src.replace("alpha", "vp"))
    elif "beta" in src:
        os.rename(src, src.replace("beta", "vs"))

### a) Visualizing the Event Kernel

- We can visualize the gradient using SeisFlows utilities
- Note that this visualization technique is only viable for SPECFEM2D
- Gradient amplitudes are stored in the output Fortran binary files
- We only plot the Vs Kernel here but the Gradient is a linear array of Vp and Vs

In [ ]:
# In this cell we'll scale the gradient (Vp and Vs only) by GTG^-1
x = read_fortran_binary("DATA/proc000000_x.bin")
z = read_fortran_binary("DATA/proc000000_z.bin")
vs_kernel = read_fortran_binary("GRADIENT/proc000000_vs_kernel.bin")

boundary_value = 2E-10  # <- change this value to adjust the colorscale

# Plot the kernel
plt.tricontourf(x, z, vs_kernel, levels=125, cmap="seismic_r", 
                vmin=-1*boundary_value, vmax=boundary_value)
plt.colorbar()

plot_sources_receivers()
plt.title("Gradient Vs")

#### Understanding the above figure

- Above we are seeing the Misfit kernel for the parameter Vs
- The misfit kernel represents sensitivity of the misfit function to changes in model parameter Vs
- Red colors tell us that the inital model is **too fast** (i.e., we need to slow down the initial model)
- Blue colors tell us that the initial model is **too slow** (i.e., we need to speed up the initial model)
- We can see large amplitudes directly around source locations (e.g., S01 and S09 at the bottom)
- These amplitudes are due to the numerical discretization of our grid, and the fact that we implement point source forces
- We can use regularization to remove the strong influence of the near-source region
- Our chosen regularization technique is smoothing

## 7) Smoothing/Regularizing Kernels

- We can use regularization to smooth out the gradient to get a more conservative picture of model update
- We use the `xsmooth_sem` executable to smooth our kernels
- We should choose smoothing length based on the shortest period of our data
- Because we know the target model, we smooth conservatively to the length scale of the checkerboards ($\Gamma$=50km)
- The SPECFEM smoothing code convolves the kernel with a Gaussian. We are allowed to choose the half-width of the Gaussian
- The relationship between half-width and full-width is $\Gamma\approx2.355\sigma$, so we choose $\sigma$=20km

 The usage of `xsmooth_sem` is given as
 ```bash
 USAGE:  mpirun -np NPROC bin/xsmooth_sem SIGMA_H SIGMA_V KERNEL_NAME INPUT_DIR OUPUT_DIR GPU_MODE
 ```
 We will need to choose values and directories to make this work
  - `SIGMA_H`: Horizontal smoothing length [m] representing the horizontal half-width of the Gaussian  
  - `SIGMA_Z`: Vertical smoothing length [m] representing the vertical half-width of the Gaussian  
  - `KERNEL_NAME`: The name of the kernel we want to smooth. Must match filename, so `proc000000_vs_kernel.bin` will correspond to kernel name `vs_kernel`  
  - `INPUT_DIR`: where SPECFEM can find the kernel files
  - `OUTPUT_DIR`: Where SPECFEM should output the SMOOTHED kernels 
  - `GPU_MODE`: Use GPU acceleration to speed up the smoothing operation (.true. or .false.)

In [ ]:
# SPECFEM Requires that some of the model files be discoverable alongside the kernels
! cp -f GRADIENT/*.bin DATA/

! mpirun -n 1 bin/xsmooth_sem 20000 20000 vs_kernel DATA/ GRADIENT/ .false.
! mpirun -n 1 bin/xsmooth_sem 20000 20000 vp_kernel DATA/ GRADIENT/ .false.

In [ ]:
# The smoothing operation has created new files with the 'smooth' tag
vs_kernel_smooth = read_fortran_binary("GRADIENT/proc000000_vs_kernel_smooth.bin")

boundary_value = 8E-11  # <- change this value to adjust the colorscale

# Plot the kernel
plt.tricontourf(x, z, vs_kernel_smooth, levels=125, cmap="seismic_r", 
                vmin=-1*boundary_value, vmax=boundary_value)
plt.colorbar()

plot_sources_receivers()
plt.title("Smoothed Gradient Vs")

#### Understanding the above figure

- Above we see the smoothed misfit kernel
- We have applied smoothing using a 2D Gaussian with half width equal to 20km
- The regularization has removed the small-scale heterogeneity present in the raw gradient
- The colors still represent the same quantities as before

## 8) Scaling the Gradient

- Again we'll need to **scale** the gradient before using it to perturb the initial model
- As in Day 3a, we'll scale the gradient by [G$^T$G]$^{-1}$
- We can then perturb the initial model by the scaled gradient to recover new synthetics

In [ ]:
# Load in the regularized kernel
vp_kernel = read_fortran_binary("GRADIENT/proc000000_vs_kernel_smooth.bin")
vs_kernel = read_fortran_binary("GRADIENT/proc000000_vs_kernel_smooth.bin")

# The Gradient is a linear vector of kernels for each parameter
gradient = np.hstack((vp_kernel, vs_kernel))

# Calculate GTG^-1
gtg = np.dot(gradient, gradient)
gtg_inverse = gtg ** -1 
print(f"GTG^-1 = {gtg_inverse:.2f}")

# Scaling the gradient by GTG^-1
gradient = gradient * gtg_inverse

## 9) Perturbing the Initial Model

- We can perturb the initial model with the gradient to generate an update model ($m_{try}$)
- We'll use this updated model to run forward simulations
- We **only** update seismic velocity Vp and Vs. However during your own research, other quantities may be updated.
- Users will need to determine what model parameters their misfit function is sensitive to
- **Objective**: Quantify misfit of the new synthetics to determine if misfit has been reduced

In [ ]:
# We will separate model files to keep things clean
! rm -rf MODEL_INIT
! mkdir MODEL_INIT
! rm -rf MODEL_01
! mkdir MODEL_01

# These files define a SPECFEM2D model. SPECFEM3D models will differ
for tag in ["x", "z", "rho", "vp", "vs", "NSPEC_ibool", "jacobian"]:
    src = f"DATA/proc000000_{tag}.bin"
    # Save the initial model to disk
    dst = f"MODEL_INIT/{os.path.basename(src)}"
    shutil.copy(src, dst)
    # Copy to Model 01 which will be overloaded by new parameters
    dst = f"MODEL_01/{os.path.basename(src)}"
    shutil.copy(src, dst)
    
# Load the initial model velocity so we can overload them
model_init_vp = read_fortran_binary("MODEL_INIT/proc000000_vp.bin")
model_init_vs = read_fortran_binary("MODEL_INIT/proc000000_vs.bin")

# Split the gradient by parameter
n = len(gradient) // 2
gradient_vp = gradient[:n]
gradient_vs = gradient[n:]

# Perturb the initial model by the gradient
model_01_vp = model_init_vp - gradient_vp
model_01_vs = model_init_vs - gradient_vs

# OVERWRITE Vp and Vs for Model 01 with perturbed model parameters
write_fortran_binary(model_01_vp, "MODEL_01/proc000000_vp.bin")
write_fortran_binary(model_01_vs, "MODEL_01/proc000000_vs.bin")

### a) Checking Model Parameters

- We can quickly determine if the update model is acceptable by checking min and max values
- We use external knowledge to quantify whether the new model is acceptable
- For example we know we cannot have negative velocity values for typical wavespeeds in Earth structure
- External geological evidence might also help constrain expected velocity values

In [ ]:
print(f"MODEL INIT Vs_min = {model_init_vs.min()}")
print(f"MODEL INIT Vs_max = {model_init_vs.max()}")
print(f"\nMODEL TRY Vs_min = {model_01_vs.min()}")
print(f"MODEL TRY Vs_max = {model_01_vs.max()}")

Already we see that our velocity values are way out of spec. We will need to manually step back our gradient. 
In a real seismic inversion, this would be algorithmically determined, but for the sake of brevity we will
simply scale back our gradient until we reach acceptable perturbation levels. Here we limit the largest value of $\alpha$ to be roughly 10\% of the initial model value

In [ ]:
# Safe guard the gradient from overreaching
max_alpha = model_init_vs.mean() * 0.1  
gradient /= gradient.max()  # reset gradient to be scaled to 1
gradient *= max_alpha  # set a new alpha value as 10% of the velocity model

# Create new updated model parameters !!! WHY IS THIS POSITIVE AND NOT NEGATIVE? !!!
model_01_vp = model_init_vp + gradient_vp
model_01_vs = model_init_vs + gradient_vs

# OVERWRITE Vp and Vs for Model 01 with perturbed model parameters
write_fortran_binary(model_01_vp, "MODEL_01/proc000000_vp.bin")
write_fortran_binary(model_01_vs, "MODEL_01/proc000000_vs.bin")

print(f"MODEL TRY Vs_min = {model_01_vs.min()}")
print(f"MODEL TRY Vs_max = {model_01_vs.max()}")

# Plot the updated model
plt.tricontourf(x, z, model_01_vs, levels=125, cmap="RdYlBu")
plt.colorbar()

plot_sources_receivers()
plt.title("Model 01 Trial 1 Vs")

## 10) Run Forward Simulations with Updated Model

- Now that we have a perturbed model (Model 01) we can use it to run forward simulations
- We can calculate misfit for each of the new synthetics 
- We can compare the overall misfit of the initial model to Model 01
- **Objective**: determine if we have reduced waveform misfit through our model perturbation

In [ ]:
# Set the correct velocity model where SPECFEM expects it
for tag in ["x", "z", "rho", "vp", "vs", "NSPEC_ibool", "jacobian"]:
    src = f"MODEL_01/proc000000_{tag}.bin"
    dst = f"DATA/proc000000_{tag}.bin"
    if os.path.exists(dst):
        os.remove(dst)
    shutil.copy(src, dst)

# Reset the simulation type to forward, since we changed it during the adjoint simulation
! seisflows sempar -P DATA/Par_file simulation_type 1
! seisflows sempar -P DATA/Par_file model gll

run_fwd_mesher_solver(ntask=10, save_synthetics="SYNTHETIC_01", save_forward=False)
quantify_misfit(misfit_file="MISFIT_01.txt", write_adjsrc=False, syn_dir="SYNTHETIC_01")

### a) Visualizing Waveform Differences


In [ ]:
# Plot an example of data, synthetic and adjoint source
t_init, d_init = np.loadtxt("SOURCE_001/SYNTHETIC/AA.S000000.BXY.semd").T
t_true, d_true = np.loadtxt("SOURCE_001/TARGET/AA.S000000.BXY.semd").T
t_m01, d_m01 = np.loadtxt("SOURCE_001/SYNTHETIC_01/AA.S000000.BXY.semd").T 

plot_waveforms(t=t_init, data=[d_init, d_true, d_m01],
               labels=["initial", "target", "update"],
               colors=["r", "k", "b"])

plt.title("Initial vs. Updated Model Synthetics")
plt.xlim([45, 145])
plt.show()

## 11) Comparing Misfit between Initial and Updated models
- Now we have two files containing misfit, one for the initial model, and one for the updated model (M01)
- Each line in the misfit file corresponds to the overall misfit for a single event
- We can compare them by taking the average of all lines in the misfit file
- Misfit: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$

!!! TOTAL MISFIT HERE !!!

In [ ]:
# Calculate misfit from each source
misfit_init, misfit_m01 = [], []

for src in sorted(glob("SOURCE_???")):
    m_init = np.loadtxt(os.path.join(src, "MISFIT_INIT.txt"))
    m_01 = np.loadtxt(os.path.join(src, "MISFIT_01.txt"))
    
    misfit_init.append(m_init.sum() / len(m_init))
    misfit_m01.append(m_01.sum() / len(m_01))
    
total_misfit_init = sum(misfit_init) / len(misfit_init)
total_misfit_m01 = sum(misfit_m01) / len(misfit_m01)

print(f"MISFIT INITIAL MODEL = {total_misfit_init}")
print(f"MISFIT UPDATED MODEL = {total_misfit_m01}")
print(f"MISFIT REDUCED? -> {bool(total_misfit_init > total_misfit_m01)}")

## 12) Final Thoughts

!!! TO DO !!!

# Automating An Inversion using SeisFlows

- Of course, now that we have gone through the rigorous manual exercise of updating a velocity model, we will show how SeisFlows automates this procedure.
- Under the hood, SeisFlows essentialy performs what we just did, scaling the velocity model, re-running forward simulations and comparing misfit values
- SeisFlows contains a built-in optimization library, which features gradient-descent, non-linear conjugate gradient (NLCG), and L-BFGS nonlinear optimization alogirthms
- This library takes care of the scaling of the gradient, automating the line search and model update
- Misfit is either calculated with the default preprocessing module, or using Pyatoa, a software designed specifically for waveform misfit quantification.

In [ ]:
! rm -rf /home/scoped/work/day_3/sfexample_2
! mkdir -p /home/scoped/work/day_3/sfexample_2
%cd /home/scoped/work/day_3/sfexample_2

! seisflows examples setup 2 -r /home/scoped/specfem2d --niter 1 --ntask 10 --nsta 10 --with_mpi
! seisflows par smooth_h 20000.
! seisflows par smooth_v 20000.
! seisflows submit 

-------------

The task will be finished when you see the log message:

```bash
////////////////////////////////////////////////////////////////////////////////
                             COMPLETE ITERATION 01                              
////////////////////////////////////////////////////////////////////////////////
2022-09-16 21:25:38 (I) | setting current iteration to: 2
```

In [ ]:
! echo "> output/ contains models, gradients, etc."
! ls output
! echo

! echo "> MODEL directories contain SPECFEM formatted model files"
! ls output/MODEL_INIT
! echo

! echo "> GRADIENT directories contain regularized misfit kernels"
! ls output/GRADIENT_01
! echo

In [ ]:
# Models/gradients can be plotted directly from the command line
! seisflows plot2d MODEL_01 vs --savefig m_01_vs.png
Image("m_01_vs.png")